In [3]:
import sys
#!{sys.executable} -m pip install pymongo
#!{sys.executable} -m pip install pyyaml
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install isoweek

#!{sys.executable} -m pip install rpy2
print(sys.executable)

/usr/bin/python3


In [25]:
from memoize import memoize

In [26]:

import os
print(os.getcwd())
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret

client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("MOBILE_NAME")]
ext_client = MongoClient(getsecret("EXT_URI"))

ext_db = ext_client[getsecret("DB_NAME")]
# Get all synced accounts and their respective users.
import urllib.request as req
import json
accounts = json.loads(req.urlopen("http://localhost:5000/synced_emails").read().decode("utf-8"))


### CONSTANTS ###
INTENSITY = "intensity"
TOTAL_TIME = "total_time"
ANDROID = "android"
HASH = "email_hash"
INSTALL = "install_id"
GOALS = "goals"
FREQ_GOALS = "freq_goals"
INFREQ_GOALS = "infreq_goals"
OTHER_TIME = "other_time"
FREQ_TIME ="freq_time"
GOAL_TIME = "goal_time"
TARGET_TIME = "target_time"
BROWSER = "browser"
BROWSER_DOMAIN = "browser_domain"
GOAL_TIMES = "goal_frequency_days"
FREQ_GOALS = "freq_goals"
INFREQ_GOALS = "infreq_goals"
BROWSER = "browser"
FREQUENCY = "frequency"
TIME = "time"
GOAL = "goal"
SPEND_LESS_TIME_LENGTH = len("custom/spend_less_time_")
AVG_FREQ_TIME = "avg_freq_time"
AVG_INFREQ_TIME = "avg_infreq_time"
AVG_GOAL_TIME = "avg_goal_time"
UNPRODUCTIVE_TIME = "unproductive_time"
PRODUCTIVE_TIME = "productive_time"
INFREQ_TIME = "infreq_time"
NUM_FREQ_GOALS = "num_freq_goals"
NUM_INFREQ_GOALS = "num_infreq_goals"
import time

@memoize
def get_sessions(user):
  output = db[user+ "_sessions"].find({"interventions": {"$exists": True}, "enabled": {"$exists": True}, "duration": {"$lt": 86400}})
  return [x for x in output]

@memoize
def get_goal_frequencies(user):
  output = ext_db[user + "_synced:goal_frequencies"].find()
  return [x for x in output]

### FUNCTIONS ###
# Get freq stats
# week -> goal -> freq
# Top-level-domain-names that are not pertinent to the application.
TLDs = ['aaa', 'abb', 'abc', 'ac', 'aco', 'ad', 'ads', 'ae', 'aeg', 'af', 'afl', 'ag', 'ai', 'aig', 'al', 'am', 'anz', 'ao', 'aol', 'app', 'aq', 'ar', 'art', 'as', 'at', 'au', 'aw', 'aws', 'ax', 'axa', 'az', 'ba', 'bar', 'bb', 'bbc', 'bbt', 'bcg', 'bcn', 'bd', 'be', 'bet', 'bf', 'bg', 'bh', 'bi', 'bid', 'bio', 'biz', 'bj', 'bm', 'bms', 'bmw', 'bn', 'bnl', 'bo', 'bom', 'boo', 'bot', 'box', 'br', 'bs', 'bt', 'buy', 'bv', 'bw', 'by', 'bz', 'bzh', 'ca', 'cab', 'cal', 'cam', 'car', 'cat', 'cba', 'cbn', 'cbs', 'cc', 'cd', 'ceb', 'ceo', 'cf', 'cfa', 'cfd', 'cg', 'ch', 'ci', 'ck', 'cl', 'cm', 'cn', 'co', 'com', 'cr', 'crs', 'csc', 'cu', 'cv', 'cw', 'cx', 'cy', 'cz', 'dad', 'day', 'dds', 'de', 'dev', 'dhl', 'diy', 'dj', 'dk', 'dm', 'dnp', 'do', 'dog', 'dot', 'dtv', 'dvr', 'dz', 'eat', 'ec', 'eco', 'edu', 'ee', 'eg', 'er', 'es', 'esq', 'et', 'eu', 'eus', 'fan', 'fi', 'fit', 'fj', 'fk', 'fly', 'fm', 'fo', 'foo', 'fox', 'fr', 'frl', 'ftr', 'fun', 'fyi', 'ga', 'gal', 'gap', 'gb', 'gd', 'gdn', 'ge', 'gea', 'gf', 'gg', 'gh', 'gi', 'gl', 'gle', 'gm', 'gmo', 'gmx', 'gn', 'goo', 'gop', 'got', 'gov', 'gp', 'gq', 'gr', 'gs', 'gt', 'gu', 'gw', 'gy', 'hbo', 'hiv', 'hk', 'hkt', 'hm', 'hn', 'hot', 'how', 'hr', 'ht', 'hu', 'ibm', 'ice', 'icu', 'id', 'ie', 'ifm', 'il', 'im', 'in', 'inc', 'ing', 'ink', 'int', 'io', 'iq', 'ir', 'is', 'ist', 'it', 'itv', 'jcb', 'jcp', 'je', 'jio', 'jlc', 'jll', 'jm', 'jmp', 'jnj', 'jo', 'jot', 'joy', 'jp', 'ke', 'kfh', 'kg', 'kh', 'ki', 'kia', 'kim', 'km', 'kn', 'kp', 'kpn', 'kr', 'krd', 'kw', 'ky', 'kz', 'la', 'lat', 'law', 'lb', 'lc', 'lds', 'li', 'lk', 'llc', 'lol', 'lpl', 'lr', 'ls', 'lt', 'ltd', 'lu', 'lv', 'ly', 'ma', 'man', 'map', 'mba', 'mc', 'md', 'me', 'med', 'men', 'mg', 'mh', 'mil', 'mit', 'mk', 'ml', 'mlb', 'mls', 'mm', 'mma', 'mn', 'mo', 'moe', 'moi', 'mom', 'mov', 'mp', 'mq', 'mr', 'ms', 'msd', 'mt', 'mtn', 'mtr', 'mu', 'mv', 'mw', 'mx', 'my', 'mz', 'na', 'nab', 'nba', 'nc', 'ne', 'nec', 'net', 'new', 'nf', 'nfl', 'ng', 'ngo', 'nhk', 'ni', 'nl', 'no', 'now', 'np', 'nr', 'nra', 'nrw', 'ntt', 'nu', 'nyc', 'nz', 'obi', 'off', 'om', 'one', 'ong', 'onl', 'ooo', 'org', 'ott', 'ovh', 'pa', 'pay', 'pe', 'pet', 'pf', 'pg', 'ph', 'phd', 'pid', 'pin', 'pk', 'pl', 'pm', 'pn', 'pnc', 'pr', 'pro', 'pru', 'ps', 'pt', 'pub', 'pw', 'pwc', 'py', 'qa', 'qvc', 're', 'red', 'ren', 'ril', 'rio', 'rip', 'ro', 'rs', 'ru', 'run', 'rw', 'rwe', 'sa', 'sap', 'sas', 'sb', 'sbi', 'sbs', 'sc', 'sca', 'scb', 'sd', 'se', 'ses', 'sew', 'sex', 'sfr', 'sg', 'sh', 'si', 'sj', 'sk', 'ski', 'sky', 'sl', 'sm', 'sn', 'so', 'soy', 'sr', 'srl', 'srt', 'st', 'stc', 'su', 'sv', 'sx', 'sy', 'sz', 'tab', 'tax', 'tc', 'tci', 'td', 'tdk', 'tel', 'tf', 'tg', 'th', 'thd', 'tj', 'tjx', 'tk', 'tl', 'tm', 'tn', 'to', 'top', 'tr', 'trv', 'tt', 'tui', 'tv', 'tvs', 'tw', 'tz', 'ua', 'ubs', 'ug', 'uk', 'uno', 'uol', 'ups', 'us', 'uy', 'uz', 'va', 'vc', 've', 'vet', 'vg', 'vi', 'vig', 'vin', 'vip', 'vn', 'vu', 'wed', 'wf', 'win', 'wme', 'wow', 'ws', 'wtc', 'wtf', 'xin', 'xxx', 'xyz', 'ye', 'you', 'yt', 'yun', 'za', 'zip', 'zm', 'zw']
def get_name(name, device):
    """
    @param name: goal name (package name for Android)
    @param device: "android" or "browser" or "browser_domain"
    @return name of goal with subdomains removed and goal annotation removed (i.e. spend_less_time)
    """
    if device == ANDROID  and name =="com.google.android.gm" or device == BROWSER and "gmail" in name:
        return "gmail"
    name = name.lower()
    if "custom" in name and device == BROWSER:
        # strip off the "custom/spend_less_time_"
        name = name[SPEND_LESS_TIME_LENGTH:]
    elif device == BROWSER:
        return name.split('/spend')[0]
    # Now we have to get juicy part of domain.
    subs = list(filter(lambda x: x != "android" and x != "google" and x != "apps" and x not in TLDs, name.split('.')))
    if device == ANDROID: 
        if len(subs) > 0:
            return subs[0]
        return name
    else:
        if len(subs) > 0:
            return subs[len(subs) - 1]    
        
def get_freq_stats(user):
    """
    Gets the frequency stats for browser.
    :param user user id 
    :return {week: {FREQ_GOALS: set(), INFREQ_GOALS: set()}}
    """
    stat = {}
    #for setting in ext_db[user + "_synced:goal_frequencies"].find():
    for setting in get_goal_frequencies():
        val = json.loads(setting["val"])
        if val["algorithm"] == "isoweek_random":
            isoWeek = int(datetime.fromtimestamp(setting["timestamp_local"]/1000 + 8 * (60 ** 2)).isocalendar()[1])
            currentIsoWeek = int(datetime.fromtimestamp(time.time()).isocalendar()[1])
            for week in range(isoWeek, currentIsoWeek + 1):
                if str(week) not in stat:
                    stat[str(week)] = {FREQ_GOALS: set([]), INFREQ_GOALS: set([])}
                if val["onweeks"][week] == 1:
                    stat[str(week)][FREQ_GOALS].add(get_name(setting["key"], BROWSER))
                else:
                    stat[str(week)][INFREQ_GOALS].add(get_name(setting["key"], BROWSER))
    return stat

/home/geza0kovacs/habitlab-data-analysis


In [5]:
%reload_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 1: Setting LC_CTYPE failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 2: Setting LC_COLLATE failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 3: Setting LC_TIME failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 4: Setting LC_MESSAGES failed, using "C" 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinte

In [6]:
# Now, let's get the mapping from productive apps to unproductive ones.
from statistics import mode, median, StatisticsError
name_to_prod = {}
with open('domain_to_productivity.json', 'r') as f:
    domain_to_prod = json.load(f)
    for domain in domain_to_prod:
        name = get_name(domain, BROWSER_DOMAIN)
        if name not in name_to_prod: 
            name_to_prod[name] = []
        name_to_prod[name].append(domain_to_prod[domain])
for name in name_to_prod:
    try:
        name_to_prod[name] = mode(name_to_prod[name])
    except StatisticsError:
        name_to_prod[name] = median(name_to_prod[name])

In [29]:
# Now, for each day, get frequency intensity
day_stats = []
import math
import moment
from datetime import datetime
counter = 0
num_users = len(accounts)
for account in accounts:
    if int(counter/num_users * 100) < int((counter + 1)/num_users * 100):
        print(str(counter/num_users * 100) + "%")
    counter += 1
    curr_date = ""
    day_data = {PRODUCTIVE_TIME: 0, UNPRODUCTIVE_TIME: 0,INTENSITY: 0, INFREQ_GOALS: set([]), TARGET_TIME:0,
                FREQ_TIME: 0, TOTAL_TIME: 0, GOAL_TIME: 0, HASH: account["_id"], GOALS: set([]), 
                FREQ_GOALS: set([]), OTHER_TIME: 0, INSTALL: '', "num_sessions": 0 }
    for user in account[ANDROID]:
        day_data[INSTALL] = user
        #for s in db[user+ "_sessions"].find({"interventions": {"$exists": True}, "enabled": {"$exists": True}, "duration": {"$lt": 86400}}):
        for s in get_sessions(user):
            if (s["duration"] <= 0):
              continue
            time = moment.unix(s["timestamp"])
            date = time.format("YYYY-MM-DD")
            if date != curr_date:
                if curr_date != "" and day_data[TOTAL_TIME] > 0 and len(day_data[GOALS]) > 0 and day_data[OTHER_TIME] > 0:
                    # Add obj and reset data
                    # take logs of time cuz time is log-norm distributed
                    # Have a more clever way of determining intensity: the % time spent on freq goals
                    for key in day_data:
                        if "domain_time" in key:
                            domain = key[:-len("_domain_time")]
                            #if domain in day_data[FREQ_GOALS] or domain in day_data[INFREQ_GOALS]:
                            #    day_data[GOAL_TIME] += day_data[key]
                    for key in day_data:
                        if "time" in key and day_data[key] > 0:
                            day_data[key]= math.log(day_data[key])
                    day_data[INTENSITY] /= day_data["num_sessions"]
                    day_stats.append(day_data)
                    isoWeek = str(datetime.fromtimestamp(moment.date(date).epoch() + 8 * (60 ** 2)).isocalendar()[1])
                    day_data = {PRODUCTIVE_TIME: 0, UNPRODUCTIVE_TIME: 0,INTENSITY: 0, INFREQ_GOALS: set([]), TARGET_TIME:0,
                                FREQ_TIME: 0, TOTAL_TIME: 0, GOAL_TIME: 0, HASH: account["_id"], GOALS: set([]), 
                                FREQ_GOALS: set([]), OTHER_TIME: 0, INSTALL: user, "num_sessions": 0, "iso": isoWeek}
                curr_date = date
            if "enabled" in s:                
                if s["enabled"]:
                    day_data[GOAL_TIME] += s["duration"]
                    day_data[GOALS].add(s["domain"])
                    if s["duration"] > 0:
                        if s["domain"] + "_domain_time" not in day_data:
                            day_data[s["domain"]+"_domain_time"] = 0
                            day_data[s["domain"] + "_num_sessions"] = 0
                            day_data[s["domain"] + "_intervention_sessions"] = 0
                        day_data[s["domain"]+"_domain_time"] += s["duration"]
                        day_data[s["domain"] + "_num_sessions"] += 1
                        if len(s["interventions"]) > 0:
                            day_data[s["domain"] +"_intervention_sessions"] += 1
                    if "frequent" in s and s["frequent"]:
                        day_data[FREQ_TIME] += s["duration"]
                        day_data[FREQ_GOALS].add(s["domain"])
                    elif "frequent" in s:
                        day_data[INFREQ_GOALS].add(s["domain"])
                elif "target" in s and s["target"]:
                    day_data[TARGET_TIME] += s["duration"]
                    day_data[OTHER_TIME] += s["duration"]
                else:
                    day_data[OTHER_TIME] += s["duration"]
            if s["enabled"]:
                day_data["num_sessions"] += 1
                if len(s["interventions"]) > 0:
                    day_data[INTENSITY] += 1
            name = get_name(s["domain"], ANDROID)
            if name in name_to_prod:
                if name_to_prod[name] > 0:
                    day_data[PRODUCTIVE_TIME] += s["duration"]
                elif name_to_prod[name] < 0:
                    day_data[UNPRODUCTIVE_TIME] += s["duration"]
            day_data[TOTAL_TIME] += (s["duration"])

0.975292587776333%
1.950585175552666%
2.990897269180754%
3.966189856957087%
4.9414824447334205%
5.981794538361508%
6.957087126137841%
7.99739921976593%
8.972691807542263%
9.947984395318596%
10.988296488946684%
11.963589076723016%
12.93888166449935%
13.97919375812744%
14.954486345903772%
15.99479843953186%
16.970091027308193%
17.945383615084527%
18.985695708712612%
19.960988296488946%
20.93628088426528%
21.976592977893368%
22.9518855656697%
23.99219765929779%
24.967490247074124%
25.942782834850455%
26.983094928478547%
27.95838751625488%
28.998699609882966%
29.9739921976593%
30.949284785435633%
31.98959687906372%
32.964889466840056%
33.940182054616386%
34.980494148244475%
35.955786736020805%
36.99609882964889%
37.971391417425224%
38.94668400520156%
39.98699609882965%
40.96228868660598%
41.93758127438231%
42.977893368010406%
43.953185955786736%
44.993498049414825%
45.96879063719116%
46.94408322496749%
47.98439531859558%
48.95968790637191%
49.93498049414825%
50.97529258777633%
51.950585175

In [8]:
import pandas as pd
prev_users= {}
# our next data set: goal_time, freq, goal, and email_hash
goal_stats_dict = {"app": [], "email": [], "time": [], "other_time": [], "frequency": [], 'intensity_day': [], 'intensity_othergoals_day': [], "other_goal_time":[], "avg_other_goal_time": []}
counter = 0
for day in day_stats:
    uhash = day[HASH]
    if uhash not in prev_users and "iso" in day:
        prev_users[uhash] = day["iso"]
    if True or "iso" in day and prev_users[uhash] != day["iso"]:
        for key in day:
            if "domain_time" in key:
                domain = key[:-len("_domain_time")]
                if not (domain in day[FREQ_GOALS] and domain in day[INFREQ_GOALS]) and (domain in day[FREQ_GOALS] or domain in day[INFREQ_GOALS]):
                    freq = "frequent" if domain in day[FREQ_GOALS] else "infrequent"
                    if math.exp(day[GOAL_TIME]) - math.exp(day[key])>= 1:
                        goal_stats_dict["intensity_othergoals_day"].append((day[INTENSITY] * day["num_sessions"] - day[domain + "_intervention_sessions"])/(day["num_sessions"]-day[domain+"_num_sessions"]))
                        goal_stats_dict["other_goal_time"].append(math.log(math.exp(day[GOAL_TIME]) - math.exp(day[key])))
                        goal_stats_dict["avg_other_goal_time"].append(math.log(math.exp(goal_stats_dict["other_goal_time"][-1])/len(day[GOALS])))
                        goal_stats_dict["intensity_day"].append(day[INTENSITY])
                        goal_stats_dict["app"].append(domain)
                        goal_stats_dict["email"].append(day[HASH])
                        goal_stats_dict["time"].append(day[key])
                        goal_stats_dict["other_time"].append(math.log(math.exp(day[GOAL_TIME]) - math.exp(day[key])))
                        goal_stats_dict["frequency"].append(freq)
goal_freq_df = pd.DataFrame(goal_stats_dict)
#goal_freq_df.to_csv("2018-09-20-within-android-goal-freq-df.csv")
print(goal_freq_df)

                                  app  \
0                  com.android.chrome   
1               com.google.android.gm   
2                   com.mattermost.rn   
3                  com.android.chrome   
4               com.google.android.gm   
5             com.netflix.mediaclient   
6               com.google.android.gm   
7               com.google.android.gm   
8             com.audible.application   
9             com.netflix.mediaclient   
10               com.linkedin.android   
11                com.twitter.android   
12              com.instagram.android   
13               com.linkedin.android   
14              com.instagram.android   
15                com.twitter.android   
16              com.instagram.android   
17                com.twitter.android   
18                com.facebook.katana   
19                  com.amazon.kindle   
20               com.linkedin.android   
21               com.linkedin.android   
22                com.twitter.android   
23              

In [9]:
import math
day_stats_dict = {"avg_goal_time": [], HASH: [], "other_time": [],  'intensity_day': [], GOAL_TIME: [] ,TOTAL_TIME: []}
prev_users= {}
for day in day_stats:
    uhash = day[HASH]
    if uhash not in prev_users and "iso" in day:
        prev_users[uhash] = {"iso": day["iso"], "count": 1}
    elif "iso" in day:
        prev_users[uhash]["count"] += 1
    if math.exp(day[TOTAL_TIME]) < 86400 and math.exp(day[OTHER_TIME]) < 86400: #True or "iso" in day and prev_users[uhash] != day["iso"] and prev_users[uhash]["count"] > 7:
        day_stats_dict[GOAL_TIME].append(day[GOAL_TIME])
        day_stats_dict[HASH].append(day[HASH])
        day_stats_dict["intensity_day"].append(day[INTENSITY])
        day_stats_dict["other_time"].append(math.log(math.exp(day[OTHER_TIME]) + math.exp(day[TARGET_TIME])))
        day_stats_dict["avg_goal_time"].append(math.log(math.exp(day[GOAL_TIME])/len(day[GOALS])))
        day_stats_dict[TOTAL_TIME].append(day[TOTAL_TIME])
day_df = pd.DataFrame(day_stats_dict)
#day_df.to_csv("2018-09-21-within_android_df.csv")
print(day_df)

      avg_goal_time                                         email_hash  \
0          7.868892  0744f7e9f4323685b86a1522b101550c9415ba2488d7e7...   
1          7.576097  0744f7e9f4323685b86a1522b101550c9415ba2488d7e7...   
2          4.135167  e8493f5e2cc89f1ca9328c889763125a1a117bd0e8f4e4...   
3          4.733270  e8493f5e2cc89f1ca9328c889763125a1a117bd0e8f4e4...   
4          5.700444  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
5          7.125015  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
6          6.467077  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
7          5.466653  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
8          6.005121  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
9          6.310827  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
10         4.649187  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
11         6.321667  739e11589479d8e6b88aec1b86ec8c4a023691db6c9d0a...   
12         4.634729  739e11589479d8e6b

In [44]:
%%R -i day_df -w 5 -h 5 --units in -r 200
summary(day_df)

 avg_goal_time     email_hash          other_time     intensity_day   
 Min.   : 0.000   Length:7768        Min.   : 2.197   Min.   :0.0000  
 1st Qu.: 6.467   Class :character   1st Qu.: 8.445   1st Qu.:0.3636  
 Median : 7.172   Mode  :character   Median : 9.457   Median :0.6154  
 Mean   : 7.056                      Mean   : 9.271   Mean   :0.6084  
 3rd Qu.: 7.829                      3rd Qu.:10.368   3rd Qu.:0.8742  
 Max.   :10.384                      Max.   :11.437   Max.   :1.0000  
   goal_time        total_time    
 Min.   : 0.000   Min.   : 3.829  
 1st Qu.: 7.691   1st Qu.: 9.308  
 Median : 8.542   Median : 9.990  
 Mean   : 8.281   Mean   : 9.848  
 3rd Qu.: 9.213   3rd Qu.:10.624  
 Max.   :11.330   Max.   :11.367  


In [45]:
%%R -i day_df -w 5 -h 5 --units in -r 200
day_df$email_hash <- factor(day_df$email_hash,  ordered=FALSE)
results <- lmer(goal_time ~ intensity_day + (1|email_hash), data = day_df)
summary(results)
class(results) <- "lmerMod"
stargazer(results,
        dep.var.labels=c("Log time spent per day", "(Intercept)"), 
        no.space=TRUE, 
        title="Time spent on a given goal. Users spend less time on goals assigned to the \"frequent\" condition.", 
        label="tab:mobile_frequency_effective", 
        keep.stat="n",
        table.placement = "tb",
        star.cutoffs = c(0.05, 0.01, 0.001)
        )
# GOAL TIME DECREASE WITH INTENSITY



% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Fri, Sep 21, 2018 - 15:49:25
\begin{table}[tb] \centering 
  \caption{Time spent on a given goal. Users spend less time on goals assigned to the "frequent" condition.} 
  \label{tab:mobile_frequency_effective} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & Log time spent per day \\ 
\hline \\[-1.8ex] 
 intensity\_day & $-$0.115$^{*}$ \\ 
  & (0.051) \\ 
  Constant & 8.146$^{***}$ \\ 
  & (0.056) \\ 
 \hline \\[-1.8ex] 
Observations & 7,768 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{1}{r}{$^{*}$p$<$0.05; $^{**}$p$<$0.01; $^{***}$p$<$0.001} \\ 
\end{tabular} 
\end{table} 


In [46]:
%Rpush day_df



/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [47]:
%%R -w 5 -h 5 --units in -r 200

summary(day_df)

 avg_goal_time     email_hash          other_time     intensity_day   
 Min.   : 0.000   Length:7768        Min.   : 2.197   Min.   :0.0000  
 1st Qu.: 6.467   Class :character   1st Qu.: 8.445   1st Qu.:0.3636  
 Median : 7.172   Mode  :character   Median : 9.457   Median :0.6154  
 Mean   : 7.056                      Mean   : 9.271   Mean   :0.6084  
 3rd Qu.: 7.829                      3rd Qu.:10.368   3rd Qu.:0.8742  
 Max.   :10.384                      Max.   :11.437   Max.   :1.0000  
   goal_time        total_time    
 Min.   : 0.000   Min.   : 3.829  
 1st Qu.: 7.691   1st Qu.: 9.308  
 Median : 8.542   Median : 9.990  
 Mean   : 8.281   Mean   : 9.848  
 3rd Qu.: 9.213   3rd Qu.:10.624  
 Max.   :11.330   Max.   :11.367  


In [48]:
%%R -w 5 -h 5 --units in -r 200
ndata = day_df
day_df$email_hash <- factor(day_df$email_hash,  ordered=FALSE)
results <- lmer(total_time ~ intensity_day + (1|email_hash), data = ndata)
summary(results)
#class(results) <- "lmerMod"
# GOAL TIME INCREASE WITH INTENSITY


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: total_time ~ intensity_day + (1 | email_hash)
   Data: ndata

REML criterion at convergence: 19384.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.3234 -0.3584  0.1616  0.5419  3.9441 

Random effects:
 Groups     Name        Variance Std.Dev.
 email_hash (Intercept) 0.8485   0.9212  
 Residual               0.5772   0.7597  
Number of obs: 7768, groups:  email_hash, 637

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)      9.54753    0.04491  913.92092 212.614   <2e-16 ***
intensity_day   -0.01725    0.03578 7471.29090  -0.482     0.63    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
intensty_dy -0.491


In [49]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)

In [50]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

# goodresult RESULT-MOBILE-SYMBIOTIC

goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(time ~ (frequency) + (1|app) + (1|email), data = goal_freq_df)
show(summary(results))
class(results) <- "lmerMod"

stargazer(results,
        dep.var.labels=c("Log time spent per day", "(Intercept)"), 
        no.space=TRUE, 
        title="Time spent on a given goal. Users spend less time on goals assigned to the \"frequent\" condition.", 
        label="tab:mobile_frequency_effective", 
        keep.stat="n",
        table.placement = "tb",
        star.cutoffs = c(0.05, 0.01, 0.001)
        )
#
#stargazer(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: time ~ (frequency) + (1 | app) + (1 | email)
   Data: goal_freq_df

REML criterion at convergence: 119941

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5327 -0.5411  0.1279  0.6641  3.6212 

Random effects:
 Groups   Name        Variance Std.Dev.
 app      (Intercept) 1.8568   1.3626  
 email    (Intercept) 0.4548   0.6744  
 Residual             2.6485   1.6274  
Number of obs: 30864, groups:  app, 748; email, 611

Fixed effects:
                    Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)        5.230e+00  6.847e-02  9.260e+02  76.383  < 2e-16 ***
frequencyfrequent -5.747e-02  1.959e-02  3.057e+04  -2.934  0.00335 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
frqncyfrqnt -0.138

% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.har

In [51]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(other_goal_time ~ (frequency) + (1|email) , data = goal_freq_df)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: other_goal_time ~ (frequency) + (1 | email)
   Data: goal_freq_df

REML criterion at convergence: 88843.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.1924 -0.3732  0.1222  0.5597  4.2669 

Random effects:
 Groups   Name        Variance Std.Dev.
 email    (Intercept) 1.2189   1.1040  
 Residual             0.9709   0.9854  
Number of obs: 30864, groups:  email, 611

Fixed effects:
                   Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       7.891e+00  4.636e-02 5.598e+02 170.222   <2e-16 ***
frequencyfrequent 3.985e-03  1.164e-02 3.032e+04   0.342    0.732    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
frqncyfrqnt -0.126


In [52]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(other_goal_time ~ (frequency) + (1|app), data = goal_freq_df)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: other_goal_time ~ (frequency) + (1 | app)
   Data: goal_freq_df

REML criterion at convergence: 101566.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.8272 -0.4318  0.1531  0.6289  2.7801 

Random effects:
 Groups   Name        Variance Std.Dev.
 app      (Intercept) 0.4323   0.6575  
 Residual             1.5316   1.2376  
Number of obs: 30864, groups:  app, 748

Fixed effects:
                   Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       8.518e+00  3.359e-02 6.278e+02 253.624   <2e-16 ***
frequencyfrequent 9.573e-04  1.436e-02 3.083e+04   0.067    0.947    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
frqncyfrqnt -0.208


In [53]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lm(other_goal_time ~ (frequency), data = goal_freq_df)
summary(results)


Call:
lm(formula = other_goal_time ~ (frequency), data = goal_freq_df)

Residuals:
    Min      1Q  Median      3Q     Max 
-8.4102 -0.5625  0.2278  0.8345  2.9211 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)        8.41024    0.01028 818.362   <2e-16 ***
frequencyfrequent -0.00139    0.01463  -0.095    0.924    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.285 on 30862 degrees of freedom
Multiple R-squared:  2.927e-07,	Adjusted R-squared:  -3.211e-05 
F-statistic: 0.009032 on 1 and 30862 DF,  p-value: 0.9243



In [54]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(other_goal_time ~ (frequency) +  (1|email) , data = goal_freq_df)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: other_goal_time ~ (frequency) + (1 | email)
   Data: goal_freq_df

REML criterion at convergence: 88843.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.1924 -0.3732  0.1222  0.5597  4.2669 

Random effects:
 Groups   Name        Variance Std.Dev.
 email    (Intercept) 1.2189   1.1040  
 Residual             0.9709   0.9854  
Number of obs: 30864, groups:  email, 611

Fixed effects:
                   Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       7.891e+00  4.636e-02 5.598e+02 170.222   <2e-16 ***
frequencyfrequent 3.985e-03  1.164e-02 3.032e+04   0.342    0.732    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
frqncyfrqnt -0.126


In [55]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200
# FREQUENCY ON OTHER GOAL TIME
goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(other_goal_time ~ (frequency) +  (1|email)  + (1|app), data = goal_freq_df)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: other_goal_time ~ (frequency) + (1 | email) + (1 | app)
   Data: goal_freq_df

REML criterion at convergence: 88078.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.2607 -0.3777  0.1117  0.5522  4.4927 

Random effects:
 Groups   Name        Variance Std.Dev.
 app      (Intercept) 0.1158   0.3403  
 email    (Intercept) 1.2163   1.1029  
 Residual             0.9316   0.9652  
Number of obs: 30864, groups:  app, 748; email, 611

Fixed effects:
                   Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       8.032e+00  5.043e-02 7.537e+02 159.276   <2e-16 ***
frequencyfrequent 3.338e-03  1.157e-02 3.030e+04   0.288    0.773    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
frqncyfrqnt -0.111


In [56]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200
# MOBILE INTENSITY OF OTHER GOALS
goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(time ~ intensity_othergoals_day + (1|app), data = goal_freq_df)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: time ~ intensity_othergoals_day + (1 | app)
   Data: goal_freq_df

REML criterion at convergence: 122036.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1666 -0.5608  0.1252  0.6701  3.3542 

Random effects:
 Groups   Name        Variance Std.Dev.
 app      (Intercept) 1.878    1.370   
 Residual             2.938    1.714   
Number of obs: 30864, groups:  app, 748

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)               5.166e+00  6.467e-02  8.100e+02  79.884   <2e-16 ***
intensity_othergoals_day -1.983e-02  3.694e-02  3.062e+04  -0.537    0.591    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
intnsty_th_ -0.343


In [57]:
%%R -i goal_freq_df -w 5 -h 5 --units in -r 200

goal_freq_df$email <- factor(goal_freq_df$email,  ordered=FALSE)
goal_freq_df$app <- factor(goal_freq_df$app, ordered=FALSE)
goal_freq_df$frequency <- factor(goal_freq_df$frequency, levels=c("infrequent", "frequent"))
#results <- lmer(time ~ intensity_othergoals_day + (1|frequency), data = goal_freq_df)
results <- lmer(other_time ~ intensity_day + (1|app), data = goal_freq_df)
summary(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: other_time ~ intensity_day + (1 | app)
   Data: goal_freq_df

REML criterion at convergence: 101555.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.8168 -0.4313  0.1538  0.6297  2.7768 

Random effects:
 Groups   Name        Variance Std.Dev.
 app      (Intercept) 0.429    0.655   
 Residual             1.531    1.237   
Number of obs: 30864, groups:  app, 748

Fixed effects:
               Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   8.463e+00  3.725e-02 9.561e+02 227.195   <2e-16 ***
intensity_day 9.393e-02  2.958e-02 3.084e+04   3.176   0.0015 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
intensty_dy -0.476
